In [ ]:
import re
import json
import openai
OPENAI_API_KEY = ''
openai.api_key = OPENAI_API_KEY


def ndesc_generator(grammar_pattern, gp_desc, group, g_desc, group_mm, examples):
    # openai.ChatCompletion.create
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
                {'role':'system', 'content':'You serve as a professional English teacher.'}, 
                {'role':'user', 'content':f"""In grammar pattern "{grammar_pattern}", {gp_desc} 
                                            Nouns exhibiting this pattern can be categorized into semantic groups.
                                            In '{group}': {g_desc}"""}, 
                {'role':'assistant', 'content':f"""{GROUP_MM} belobg to '{group}'.
                and here are some example sentences: {examples}"""}, 
                {"role": "user",
                "content": f"""The noun group in {grammar_pattern} of '{group}' indicates [MASK]."""}
        ],
        temperature=0,
        max_tokens=1500,
    )
    # generalize, induce
    response_text = response.choices[0].message.content
    return response_text

def extract_partial_sentence(text):
    pattern = r'indicates(.*)'
    match = re.search(pattern, text)

    if match:
        partial_sentence = match.group(1).strip()
        return partial_sentence
    else:
        return None

def subject_extractor(n_desc):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "user",
                "content": f"""Task: Extract the subject of sentence.
                example:
                Q: someone sympathizes with someone else.
                A: someone
                Q: {n_desc}
                A: [MASK]
                """
            },
        ],
        temperature=0,
        max_tokens=1500,
    )
    # generalize, induce
    subject = response.choices[0].message.content
    return subject



NDESC = ndesc_generator(PATTERN, GP_DESC, GROUP, G_DESC, GROUP_MM, EXAMPLE)
_ = extract_partial_sentence(NDESC)
SC = subject_extractor(_)

In [6]:
import re
import nltk
import time
import pprint
import numpy as np
import pandas as pd
from string import *
import transformers as ppb
from collections import Counter

import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
from spacy import displacy

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.cluster import adjusted_rand_score, silhouette_score


## Elbow Plot
def plot_elbow_method(embedding):
  Sum_of_squared_distances = []
  K = range(1,6) #1-5
  for k in K:
      km = KMeans(n_clusters=k)
      km = km.fit(embedding)
      labels = km.labels_
      print(labels)
      Sum_of_squared_distances.append(km.inertia_)
  plt.plot(K, Sum_of_squared_distances, 'bx-')
  plt.xlabel('k')
  plt.ylabel('Sum_of_squared_distances')
  plt.title('Elbow Method For Optimal k')
  plt.show()

## Scatter Plot-2D
def plot_graph(clusters, umap_data):
  result = pd.DataFrame(umap_data, columns=['x', 'y'])
  result['labels'] = clusters
  plt.figure(figsize=(10,10))
  sns.scatterplot(
      x="x", y="y",
      hue="labels",
      palette='Spectral',
      data=result,
      legend="full")

## PCA and Kmeans
def get_cluster_kmeans(vectorize, num_clusters, r_state=100):
  km = KMeans(n_clusters = num_clusters, random_state=r_state )
  km.fit(vectorize)
  cluster_list = km.labels_.tolist()
  return cluster_list

def pca_reduction(similarity_matrix, n_components):
  one_min_sim = 1 - similarity_matrix
  pca = PCA(n_components=10)
  pos = pca.fit_transform(one_min_sim)
  x_pos, y_pos = pos[:, 0], pos[:, 1]
  return (x_pos, y_pos)

In [ ]:
from sentence_transformers import SentenceTransformer


# sample test
labels = ["topic", "subject", "clothes", "attire", "taste", "smell", "sound"]
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')
corpus_embeddings = embedder.encode(labels)

# (Optional) Normalize the embeddings to unit length
normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)


def plot_elbow_method(embedding):

  Sum_of_squared_distances = []
  K = range(60, 75) #1-5
  for k in K:
      km = KMeans(n_clusters=k)
      km = km.fit(embedding)


      label_to_tags = {}
      # Iterate over labels and corresponding tags
      for label, tag in zip(km.labels_, labels):
          if label not in label_to_tags:
              label_to_tags[label] = []
          label_to_tags[label].append(tag)

      # Print the mapping
      for label, tag_list in sorted(label_to_tags.items()):
          print(f"{label}: {', '.join(tag_list)}")

      Sum_of_squared_distances.append(km.inertia_)
  plt.plot(K, Sum_of_squared_distances, 'bx-')
  plt.xlabel('k')
  plt.ylabel('Sum_of_squared_distances')
  plt.title('Elbow Method For Optimal k')
  plt.show()


plot_elbow_method(corpus_embeddings)